In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import multiprocessing

# eQTL数据处理

In [ ]:
cis_df_all_eQTL = pd.read_csv('/CIMA/Result/20250108_cis_eQTL_all.csv',index_col=0)
cis_df_all_eQTL_sig = cis_df_all_eQTL[cis_df_all_eQTL['study_wise_qval'] < 0.05].copy()

In [ ]:
#输出所有显著eSNP的基因分型
genotype_df = pd.read_parquet('/CIMA/Data/413_sample_genotype.parquet')
genotype_df.loc[cis_df_all_eQTL_sig['variant_id'].unique(),:].T.to_csv('/CIMA/Data/celltype_independent/genotype_sigeSNP.csv')

In [ ]:
# 创建一个空的DataFrame来存储比例结果
overlap_count = pd.DataFrame(index=cis_df_all_eQTL_sig['celltype'].unique(), columns=cis_df_all_eQTL_sig['celltype'].unique())
overlap_count_with_diff_topsnp = pd.DataFrame(index=cis_df_all_eQTL_sig['celltype'].unique(), columns=cis_df_all_eQTL_sig['celltype'].unique())

for celltype_A in cis_df_all_eQTL_sig['celltype'].unique():
    df_merge = pd.DataFrame()
    for celltype_B in cis_df_all_eQTL_sig['celltype'].unique():
        if celltype_A != celltype_B:
            use_df_A = cis_df_all_eQTL_sig[cis_df_all_eQTL_sig['celltype'] == celltype_A][['phenotype_id','variant_id']]
            series_A = pd.Series(use_df_A['variant_id'].values, index=use_df_A['phenotype_id'].values)
            use_df_B = cis_df_all_eQTL_sig[cis_df_all_eQTL_sig['celltype'] == celltype_B][['phenotype_id','variant_id']]
            series_B = pd.Series(use_df_B['variant_id'].values, index=use_df_B['phenotype_id'].values)

            # 合并两个 Series，保留共享的索引，并设置列名
            df = pd.concat([series_A, series_B], axis=1, join='inner').reset_index()
            df.columns = ['phenotype','topSNP_A', 'topSNP_B']
            df['celltype_A'] = celltype_A
            df['celltype_B'] = celltype_B
            df = df[['celltype_A','celltype_B','phenotype','topSNP_A','topSNP_B']]
            
            # 统计合并后的行数
            initial_row_count = len(df)
            overlap_count.loc[celltype_A,celltype_B] = initial_row_count

            # 去除 topSNP_A 等于 topSNP_B 的行
            df_filtered = df[df['topSNP_A'] != df['topSNP_B']]

            # 统计去除后的行数
            filtered_row_count = len(df_filtered)
            overlap_count_with_diff_topsnp.loc[celltype_A,celltype_B] = filtered_row_count
            df_merge = pd.concat([df_merge,df_filtered]).reset_index(drop = True)
    df_merge.to_csv(f'/CIMA/Data/celltype_independent/eQTL_df/{celltype_A}.csv')    

In [5]:
df_merge

,celltype_A,celltype_B,phenotype,topSNP_A,topSNP_B
0,Transitional_NK_GZMK,Atypical_Bm_ITGAX,SSU72,chr1_1541943,chr1_1538787
1,Transitional_NK_GZMK,Atypical_Bm_ITGAX,SLC35E2B,chr1_1651653,chr1_1646352
2,Transitional_NK_GZMK,Atypical_Bm_ITGAX,SLC35E2A,chr1_1618982,chr1_1660318
3,Transitional_NK_GZMK,Atypical_Bm_ITGAX,CAPZB,chr1_19340105,chr1_19354899
4,Transitional_NK_GZMK,Atypical_Bm_ITGAX,PINK1-AS,chr1_20647379,chr1_20636214
...,...,...,...,...,...
19308,Transitional_NK_GZMK,Terminal_NK_dim_CD160neg,GRAMD4,chr22_46588368,chr22_46602247
19309,Transitional_NK_GZMK,Terminal_NK_dim_CD160neg,AL118516.1,chr22_46762282,chr22_46762547
19310,Transitional_NK_GZMK,Terminal_NK_dim_CD160neg,MLC1,chr22_50162190,chr22_50092222
19311,Transitional_NK_GZMK,Terminal_NK_dim_CD160neg,TYMP,chr22_50524928,chr22_50525111


In [ ]:
overlap_count.to_csv('/CIMA/Result/summary/20250207_sig_eGene_celltype_overlapcount.csv')
overlap_count_with_diff_topsnp.to_csv('/CIMA/Result/summary/20250207_sig_eGene_withdifftopsnp_celltype_overlapcount.csv')

# caQTL数据处理

In [ ]:
cis_df_all_caQTL = pd.read_csv('/CIMA/Result/20250108_cis_caQTL_all.csv',index_col=0)
cis_df_all_caQTL_sig = cis_df_all_caQTL[cis_df_all_caQTL['study_wise_qval'] < 0.05].copy()

In [ ]:
#输出所有显著eSNP的基因分型
genotype_df = pd.read_parquet('/CIMA/Data/413_sample_genotype.parquet')
genotype_df.loc[cis_df_all_caQTL_sig['variant_id'].unique(),:].T.to_csv('/CIMA/Data/celltype_independent/genotype_sigcaSNP.csv')

In [ ]:
# 创建一个空的DataFrame来存储比例结果
overlap_count = pd.DataFrame(index=cis_df_all_caQTL_sig['celltype'].unique(), columns=cis_df_all_caQTL_sig['celltype'].unique())
overlap_count_with_diff_topsnp = pd.DataFrame(index=cis_df_all_caQTL_sig['celltype'].unique(), columns=cis_df_all_caQTL_sig['celltype'].unique())

for celltype_A in cis_df_all_caQTL_sig['celltype'].unique():
    df_merge = pd.DataFrame()
    for celltype_B in cis_df_all_caQTL_sig['celltype'].unique():
        if celltype_A != celltype_B:
            use_df_A = cis_df_all_caQTL_sig[cis_df_all_caQTL_sig['celltype'] == celltype_A][['phenotype_id','variant_id']]
            series_A = pd.Series(use_df_A['variant_id'].values, index=use_df_A['phenotype_id'].values)
            use_df_B = cis_df_all_caQTL_sig[cis_df_all_caQTL_sig['celltype'] == celltype_B][['phenotype_id','variant_id']]
            series_B = pd.Series(use_df_B['variant_id'].values, index=use_df_B['phenotype_id'].values)

            # 合并两个 Series，保留共享的索引，并设置列名
            df = pd.concat([series_A, series_B], axis=1, join='inner').reset_index()
            df.columns = ['phenotype','topSNP_A', 'topSNP_B']
            df['celltype_A'] = celltype_A
            df['celltype_B'] = celltype_B
            df = df[['celltype_A','celltype_B','phenotype','topSNP_A','topSNP_B']]
            
            # 统计合并后的行数
            initial_row_count = len(df)
            overlap_count.loc[celltype_A,celltype_B] = initial_row_count

            # 去除 topSNP_A 等于 topSNP_B 的行
            df_filtered = df[df['topSNP_A'] != df['topSNP_B']]

            # 统计去除后的行数
            filtered_row_count = len(df_filtered)
            overlap_count_with_diff_topsnp.loc[celltype_A,celltype_B] = filtered_row_count
            df_merge = pd.concat([df_merge,df_filtered]).reset_index(drop = True)
    df_merge.to_csv(f'/CIMA/Data/celltype_independent/caQTL_df/{celltype_A}.csv')    

In [ ]:
overlap_count.to_csv('/CIMA/Result/summary/20250207_sig_caPeak_celltype_overlapcount.csv')
overlap_count_with_diff_topsnp.to_csv('/CIMA/Result/summary/20250207_sig_caPeak_withdifftopsnp_celltype_overlapcount.csv')